# MLFN Model

## Loading images names, identities and cameras

In [25]:
import torchreid
import torch
import json
import re

newInfo = []

with open('dataFeatures.json') as json_file:
    data = json.load(json_file)
    for p in data['images']:
        m = re.search('_p(.+?).png', p['imageName'])
        if m:
            found = m.group(1)
        newInfo.append([p['imageName'], found, p['camera']])

In [26]:
#Saving only different identities, to know the number of identities in the dataset.
P = []
for data in newInfo:
    if data[1] not in P:
        P.append(data[1])
        
print(P)

['476', '84', '236', '21', '15', '38', '47', '6', '254', '62', '197', '474', '126', '157', '181', '422', '195', '332', '190', '477', '200', '213', '322', '152', '194', '95', '486', '202', '291', '48', '104', '192', '239', '68', '176', '112', '108', '64', '199', '77', '7', '296', '1', '408', '78', '284', '289', '65', '401', '133', '223', '381', '107', '186', '311', '334', '234', '30', '506', '319', '18', '286', '164', '321', '117', '426', '46', '8', '383', '69', '281', '67', '430', '189', '395', '497', '238', '363', '412', '119', '43', '330', '399', '211', '20', '344', '427', '500', '2', '436', '35', '377', '320', '210', '299', '145', '290', '70', '315', '459', '100', '360', '447', '277', '463', '96', '245', '266', '386', '102', '120', '165', '359', '280', '39', '487', '423', '111', '103', '306', '141', '278', '255', '416', '156', '446', '448', '71', '171', '143', '429', '83', '243', '51', '339', '484', '457', '397', '150', '129', '303', '140', '300', '178', '175', '105', '9', '81', '48

In [27]:
# Appending to each id, a zero-based id in order for the algorithm to function properly.

idPairs = []
for i in range(len(P)):
    idPairs.append([i, P[i]])
    
print(idPairs)

[[0, '476'], [1, '84'], [2, '236'], [3, '21'], [4, '15'], [5, '38'], [6, '47'], [7, '6'], [8, '254'], [9, '62'], [10, '197'], [11, '474'], [12, '126'], [13, '157'], [14, '181'], [15, '422'], [16, '195'], [17, '332'], [18, '190'], [19, '477'], [20, '200'], [21, '213'], [22, '322'], [23, '152'], [24, '194'], [25, '95'], [26, '486'], [27, '202'], [28, '291'], [29, '48'], [30, '104'], [31, '192'], [32, '239'], [33, '68'], [34, '176'], [35, '112'], [36, '108'], [37, '64'], [38, '199'], [39, '77'], [40, '7'], [41, '296'], [42, '1'], [43, '408'], [44, '78'], [45, '284'], [46, '289'], [47, '65'], [48, '401'], [49, '133'], [50, '223'], [51, '381'], [52, '107'], [53, '186'], [54, '311'], [55, '334'], [56, '234'], [57, '30'], [58, '506'], [59, '319'], [60, '18'], [61, '286'], [62, '164'], [63, '321'], [64, '117'], [65, '426'], [66, '46'], [67, '8'], [68, '383'], [69, '69'], [70, '281'], [71, '67'], [72, '430'], [73, '189'], [74, '395'], [75, '497'], [76, '238'], [77, '363'], [78, '412'], [79, '11

In [28]:
#Updating in the original dataset, the old identities with the new ones.
for data in newInfo:
    if data[1] in P:
        data[1] = P.index(data[1])

In [29]:
#Idenfiying all the cameras id's existing in the dataset.
P = []
for data in newInfo:
    if data[2] not in P:
        P.append(data[2])
        
print(P)

[2, 1, 3]


In [30]:
#Saving pairs of old and new cameras ids, but the new ones zero based as the library requires.
idPairs = []
for i in range(len(P)):
    idPairs.append([i, P[i]])
    
print(idPairs)

[[0, 2], [1, 1], [2, 3]]


In [31]:
#Updating the old data with the new one.
for data in newInfo:
    if data[2] in P:
        data[2] = P.index(data[2])

In [32]:
#Transforming the data into tuples as the model requires it.
import random

for i in range(len(newInfo)):
    newInfo[i] = tuple(newInfo[i])
    
random.seed(20)
random.shuffle(newInfo)

In [33]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import sys
import os
import os.path as osp

from torchreid.data import ImageDataset

trainData = newInfo[:int(len(newInfo)*0.5)]
testData = newInfo[int(len(newInfo)*0.5):]

class NewDataset(ImageDataset):
    dataset_dir = 'new_dataset'

    def __init__(self, root='', **kwargs):
        self.root = osp.abspath(osp.expanduser(root))
        self.dataset_dir = osp.join(self.root, self.dataset_dir)

        # All you need to do here is to generate three lists,
        # which are train, query and gallery.
        # Each list contains tuples of (img_path, pid, camid),
        # where
        # - img_path (str): absolute path to an image.
        # - pid (int): person ID, e.g. 0, 1.
        # - camid (int): camera ID, e.g. 0, 1.
        # Note that
        # - pid and camid should be 0-based.
        # - query and gallery should share the same pid scope (e.g.
        #   pid=0 in query refers to the same person as pid=0 in gallery).
        # - train, query and gallery share the same camid scope (e.g.
        #   camid=0 in train refers to the same camera as camid=0
        #   in query/gallery).
        train = trainData
        query = testData[:int(len(testData)*0.2)]
        gallery = testData[int(len(testData)*0.2):]

        super(NewDataset, self).__init__(train, query, gallery, **kwargs)

In [35]:
import torchreid
torchreid.data.register_image_dataset('new_dataset', NewDataset)

In [36]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='new_dataset',
    height=256,
    width=128,
    combineall=False,
    num_instances=4,
    train_sampler='RandomIdentitySampler' # this is important
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded NewDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   457 |    17078 |         3
  query    |   361 |     3415 |         3
  gallery  |   472 |    13663 |         3
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded NewDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   457 |    17078 |         3
  query    |   361 |     3415 |         3
  gallery  |   472 |    13663 |         3
  -------------

In [37]:

model = torchreid.models.build_model(
    name='mlfn',
    num_classes=datamanager.num_train_pids,
    loss='triplet'
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

In [38]:
engine = torchreid.engine.ImageTripletEngine(
    datamanager, model, optimizer, margin=0.3,
    weight_t=0.7, weight_x=1, scheduler=scheduler
)

In [16]:
engine.run(
    max_epoch=10,
    save_dir='log/',
    print_freq=5
)

##### Evaluating new_dataset (source) #####
Extracting features from query set ...
Done, obtained 8156-by-1024 matrix
Extracting features from gallery set ...
Done, obtained 34156-by-1024 matrix
Speed: 0.1373 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 94.4%
CMC curve
Rank-1  : 91.2%
Rank-5  : 100.0%
Rank-10 : 100.0%
Rank-20 : 100.0%


## Features extraction and saving using the trained model

In [16]:
from torchreid.utils import FeatureExtractor

In [17]:
torchreid.models.show_avai_models()

['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0']


In [18]:
extractor = FeatureExtractor(
    model_name='mlfn',
    model_path='log/model/model.pth.tar-10',
    device='cuda'
)

Model: mlfn
- params: 32,473,024
- flops: 2,771,421,376
Successfully loaded pretrained weights from "log/model/model.pth.tar-10"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [19]:
image_list = []
for data in newInfo:
    image = {'imageName' : data[0]}
    image_list.append(image)

In [20]:
for data in image_list:
    features = extractor(data['imageName'])
    data['features'] = features.flatten().squeeze().tolist()

print(image_list[0])

{'imageName': '/home/dell_mserver_01/Documentos/Luis/PyTorch-YOLOv3/labeledNew/V224038_p476.png', 'features': [0.3456681966781616, 0.0, 0.4688979685306549, 0.12035153806209564, 0.19953694939613342, 0.0, 0.2690369486808777, 0.09252633154392242, 0.8312305212020874, 0.0649939551949501, 0.00464617321267724, 0.0, 0.13633021712303162, 0.317126601934433, 0.3984532952308655, 0.12952743470668793, 0.2747224271297455, 0.3542264699935913, 0.0, 0.45716774463653564, 0.0, 0.05352328345179558, 0.24331888556480408, 0.0, 0.08597107976675034, 0.12340987473726273, 0.17044711112976074, 1.247480869293213, 0.003363365773111582, 0.3195626735687256, 0.05143032968044281, 0.06677894294261932, 0.7290876507759094, 0.09711004048585892, 0.2643001079559326, 0.0015615365700796247, 0.12429630011320114, 0.5434496998786926, 0.1503916233778, 0.1640179455280304, 0.0, 0.7744327783584595, 0.20371639728546143, 0.5511561632156372, 0.24454419314861298, 0.0, 0.2507380247116089, 0.0700693354010582, 0.0, 0.20229344069957733, 0.809

In [21]:
originalJson = None

with open('dataFeatures.json') as json_file:
    originalJson = json.load(json_file)   

In [22]:
for image in image_list:
    for imageO in originalJson['images']:
        if image['imageName'] == imageO['imageName']:
            imageO['features'] = image['features']
            
print(originalJson['images'][0])

{'features': [0.3456681966781616, 0.0, 0.4688979685306549, 0.12035153806209564, 0.19953694939613342, 0.0, 0.2690369486808777, 0.09252633154392242, 0.8312305212020874, 0.0649939551949501, 0.00464617321267724, 0.0, 0.13633021712303162, 0.317126601934433, 0.3984532952308655, 0.12952743470668793, 0.2747224271297455, 0.3542264699935913, 0.0, 0.45716774463653564, 0.0, 0.05352328345179558, 0.24331888556480408, 0.0, 0.08597107976675034, 0.12340987473726273, 0.17044711112976074, 1.247480869293213, 0.003363365773111582, 0.3195626735687256, 0.05143032968044281, 0.06677894294261932, 0.7290876507759094, 0.09711004048585892, 0.2643001079559326, 0.0015615365700796247, 0.12429630011320114, 0.5434496998786926, 0.1503916233778, 0.1640179455280304, 0.0, 0.7744327783584595, 0.20371639728546143, 0.5511561632156372, 0.24454419314861298, 0.0, 0.2507380247116089, 0.0700693354010582, 0.0, 0.20229344069957733, 0.8091312646865845, 0.355815052986145, 1.146215796470642, 0.21096526086330414, 0.4934813678264618, 0.0

In [23]:
print(len(originalJson['images'][0]['features']))

1024


In [24]:
with open('dataFeaturesMLFN.json', 'w') as outfile:
    json.dump(originalJson, outfile)

## Cleaning memory

In [24]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()